In [20]:
import xgboost
import pandas as pd
import time

In [21]:
df = pd.read_csv(r'E:\Desktop\玉山\母體.csv')

In [22]:
df

,ALERT_KEY,CUST_ID,DATEKEY,RISK_RANK,OCCUPATION_CODE,TOTAL_ASSET,AGE,SAR_FLAG
0,352249,82595ac69158ae08d34156784bdec0d9e2ca5b242b6d2a...,365,1,19.0,1465816,7,NaN
1,352253,b212d14cb35676926682b2cf849e295d948888f556c07e...,365,1,2.0,98177,2,NaN
2,352254,e5b0002791c7852644a2730abeaa893cdf14a072ef7812...,365,1,19.0,2052922,7,NaN
3,352280,74214c478dc6519fbefe4bc31693865bdcd698ab974b64...,365,3,15.0,201906,5,NaN
4,352282,0340e7611f0d82c3cb87e6194fa14bb2ccf8afbf1b3418...,365,1,12.0,7450,5,NaN
...,...,...,...,...,...,...,...,...
25746,352123,b600c0720096c241f5aec16b096c6a353492eee8a5855a...,364,1,17.0,12207,2,0.0
25747,352124,06c5ea5ccda4dfd8839c0dec8646fb3071d2c02883ef8a...,364,1,17.0,259985,4,0.0
25748,352125,0c2dc5fedc3689abf5ff4be14fe8fea8d23d22068297c2...,364,3,19.0,928963,3,0.0
25749,352128,ffe5f2bd86ecfd7d0a0f4c4b76dd5c312649be95eb8b42...,364,3,19.0,21647,4,0.0


In [23]:
CUSTINFO_FEATURES = pd.read_csv(r'E:\Desktop\玉山\CUSTINFO_FEATURES.csv')
DP_FEATURES = pd.read_csv(r'E:\Desktop\玉山\DP_FEATURES.csv')
CCBA_FEATURES = pd.read_csv(r'E:\Desktop\玉山\CCBA_FEATURES.csv')
REMIT_FEATURES = pd.read_csv(r'E:\Desktop\玉山\REMIT_FEATURES.csv')
CDTX_FEATURES = pd.read_csv(r'E:\Desktop\玉山\CDTX_FEATURES.csv')

In [32]:
CCBA_FEATURES.columns = ['CUST_ID', 'LUPAY', 'DATEKEY', 'CYCAM', 'USGAM', 'CLAMT', 'CSAMT',
       'INAMT', 'CUCSM', 'CUCAH']

In [33]:
CCBA_FEATURES.head()

,CUST_ID,LUPAY,DATEKEY,CYCAM,USGAM,CLAMT,CSAMT,INAMT,CUCSM,CUCAH
0,0015cc01f553512007705edf855baa16f1b57a16fb107e...,3920.0,0,134614.0,32316.0,0.0,0.0,3148.0,16576.0,0.0
1,0015cc01f553512007705edf855baa16f1b57a16fb107e...,3920.0,1,134614.0,32316.0,0.0,0.0,3148.0,16576.0,0.0
2,0015cc01f553512007705edf855baa16f1b57a16fb107e...,3920.0,2,134614.0,32316.0,0.0,0.0,3148.0,16576.0,0.0
3,0015cc01f553512007705edf855baa16f1b57a16fb107e...,3920.0,3,134614.0,32316.0,0.0,0.0,3148.0,16576.0,0.0
4,0015cc01f553512007705edf855baa16f1b57a16fb107e...,3920.0,4,134614.0,32316.0,0.0,0.0,3148.0,16576.0,0.0


In [24]:
class renamer():
    def __init__(self):
        self.d = dict()
    def __call__(self, x):
        if x not in self.d:
            self.d[x] =  0
            return x
        else:
            self.d[x] += 1
            return "%s_%d" % (x, self.d[x])

In [35]:
t0 = time.time()

df_combined = pd.concat([df.reset_index(drop=True)
                          , CUSTINFO_FEATURES.set_index(['CUST_ID', 'DATEKEY']).reindex(df[['CUST_ID','DATEKEY']].values).reset_index(drop=True)
                          , DP_FEATURES.set_index(['cust_id', 'datekey']).reindex(df[['CUST_ID','DATEKEY']].values).reset_index(drop=True)
                          , CCBA_FEATURES.set_index(['CUST_ID', 'DATEKEY']).reindex(df[['CUST_ID','DATEKEY']].values).reset_index(drop=True)
                          , REMIT_FEATURES.set_index(['CUST_ID', 'DATEKEY']).reindex(df[['CUST_ID','DATEKEY']].values).reset_index(drop=True)
                          , CDTX_FEATURES.set_index(['CUST_ID', 'DATEKEY']).reindex(df[['CUST_ID','DATEKEY']].values).reset_index(drop=True)
                        ], axis=1)

df_combined = df_combined.rename(columns=renamer())
t1 = time.time()
print("Running time of: %.0f sec" %(t1 - t0))

Running time of: 49 sec


In [36]:
df_combined

,ALERT_KEY,CUST_ID,DATEKEY,RISK_RANK,OCCUPATION_CODE,TOTAL_ASSET,AGE,SAR_FLAG,ALERT_TIMES_1D,ALERT_TIMES_3D,...,TW_TWD_DAY_DIFF_PER_60D,TW_FCD_AMT_DIFF_PER_60D,TW_FCD_CNT_DIFF_PER_60D,TW_FCD_DAY_DIFF_PER_60D,FN_TWD_AMT_DIFF_PER_60D,FN_TWD_CNT_DIFF_PER_60D,FN_TWD_DAY_DIFF_PER_60D,FN_FCD_AMT_DIFF_PER_60D,FN_FCD_CNT_DIFF_PER_60D,FN_FCD_DAY_DIFF_PER_60D
0,352249,82595ac69158ae08d34156784bdec0d9e2ca5b242b6d2a...,365,1,19.0,1465816,7,NaN,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,352253,b212d14cb35676926682b2cf849e295d948888f556c07e...,365,1,2.0,98177,2,NaN,1,2,...,0.500000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,352254,e5b0002791c7852644a2730abeaa893cdf14a072ef7812...,365,1,19.0,2052922,7,NaN,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,352280,74214c478dc6519fbefe4bc31693865bdcd698ab974b64...,365,3,15.0,201906,5,NaN,2,3,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,352282,0340e7611f0d82c3cb87e6194fa14bb2ccf8afbf1b3418...,365,1,12.0,7450,5,NaN,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25746,352123,b600c0720096c241f5aec16b096c6a353492eee8a5855a...,364,1,17.0,12207,2,0.0,2,3,...,-0.078431,0.0,0.0,0.0,-0.484677,0.208955,0.533333,0.000000,0.000000,0.000000
25747,352124,06c5ea5ccda4dfd8839c0dec8646fb3071d2c02883ef8a...,364,1,17.0,259985,4,0.0,2,3,...,-0.036364,0.0,0.0,0.0,-198.010822,-6.250000,-1.875000,0.000000,0.000000,0.000000
25748,352125,0c2dc5fedc3689abf5ff4be14fe8fea8d23d22068297c2...,364,3,19.0,928963,3,0.0,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25749,352128,ffe5f2bd86ecfd7d0a0f4c4b76dd5c312649be95eb8b42...,364,3,19.0,21647,4,0.0,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
df_combined.to_csv(r'E:\Desktop\玉山\FINAL_FEATURES.csv', index = False)